---
title: Aggregate climate data to DHIS2 organisation units
short_title: Org unit aggregation
---

In this notebook we will show how to load daily climate data from NetCDF using [earthkit](https://ecmwf.github.io/earthkit-website/) and aggregate temperature and precipitation climate variables to DHIS2 organisation units. 

In [2]:
import geopandas as gpd
import earthkit.data
from earthkit import transforms
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json

## Loading the data

Our sample NetCDF file contains daily temperature and precipitation data for Sierra Leone in July 2025. Let's load the file using earthkit:

In [3]:
file = "../data/era5-daily-temp-precip-july-2025-sierra-leone.nc"
data = earthkit.data.from_source("file", file)

See [more examples](https://earthkit-data.readthedocs.io/en/latest/examples/) for how you can load data with eartkit, or see the video below.

:::{iframe} https://www.youtube.com/embed/no01ovW1pF8
:width: 100%
How to get data with earthkit
:::

To more easily work with and display the contents of the dataset we can convert it to an [xarray](https://xarray.dev). It shows that the file includes 3 dimensions (`latitude`, `longitude` and `valid_time`) and two data variables `t2m` (temperature at 2m above sea level), and `tp` (total precipitation). The data source is European Centre for Medium-Range Weather Forecasts ([ECMWF](https://www.ecmwf.int)). 

In [4]:
data_array = data.to_xarray()
data_array

<xarray.Dataset> Size: 41kB
Dimensions:     (valid_time: 30, latitude: 13, longitude: 13)
Coordinates:
    number      int64 8B ...
  * latitude    (latitude) float64 104B 9.918 9.668 9.418 ... 7.418 7.168 6.917
  * longitude   (longitude) float64 104B -13.3 -13.05 -12.8 ... -10.55 -10.3
  * valid_time  (valid_time) datetime64[ns] 240B 2025-07-01 ... 2025-07-30
Data variables:
    t2m         (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(30, 13, 13), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(30, 13, 13), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-11T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

## Loading the organisation units

We next use geopandas to load our organisation units that we've [downloaded from DHIS2 as a GeoJSON file](../org-units/download-manual.md): 

In [5]:
district_file = "../data/sierra-leone-districts.geojson"
org_units = gpd.read_file(district_file)

The GeoJSON file contains the boundaries of 13 named organisation units in Sierra Leone. For the aggregation, we are particularly interested in the `id` and the `geometry` (polygon) of the org unit:

In [6]:
org_units

,type,id,name,hasCoordinatesDown,hasCoordinatesUp,level,grandParentParentGraph,grandParentId,parentGraph,parentId,parentName,dimensions,weight,geometry
0,Polygon,O6uvpzGd5pu,Bo,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,Polygon,fdc6uOvgoji,Bombali,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,MultiPolygon,lc3eMKXaEfw,Bonthe,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,Polygon,jUb8gELQApl,Kailahun,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,MultiPolygon,PMa2VCrupOd,Kambia,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,Polygon,kJq2mPyFEHo,Kenema,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,Polygon,qhqAxPSTUXp,Koinadugu,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Polygon,Vth0fbpFcsO,Kono,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,MultiPolygon,jmIPBj66vD6,Moyamba,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,MultiPolygon,TEQlaapDQoK,Port Loko,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


## Aggregating the data to organisation units

To aggregate the data to the org unit features we use the `spatial.reduce` function of [earthkit-transforms](https://earthkit-transforms.readthedocs.io). We keep the daily period type and only aggregate the data spatially to the org unit features. 

Since our climate data variables need to be aggregated using different statistics, we do separate aggregations for each variable. 

### Temperature

To aggregate the temperature variable, we extract the temperature or `t2m` variable, and tell the `spatial.reduce` function to aggregate the data to our organisation units `org_units`. We set `mask_dim='id'` to specify that we want one aggregated value for every unique value in the organisation unit `id` column. Finally, we set `how='mean'` so that we get the average temperature of all gridded values that land inside an organisation unit. 

In [7]:
temp = data_array['t2m']
agg_temp = transforms.spatial.reduce(temp, org_units, mask_dim='id', how='mean')
agg_temp

<xarray.DataArray 't2m' (id: 13, valid_time: 30)> Size: 2kB
array([[297.2219 , 297.93936, 297.7815 , 297.493  , 297.60397, 297.40866,
        297.5076 , 297.4697 , 296.60388, 297.48337, 298.7655 , 298.0846 ,
        296.92996, 297.06808, 298.08807, 298.46387, 296.75964, 296.464  ,
        298.5348 , 298.1179 , 297.4404 , 297.49707, 296.39035, 296.57297,
        297.44522, 296.6952 , 297.32858, 297.13968, 298.27234, 298.66013],
       [297.70157, 297.66763, 298.27267, 297.69855, 297.9221 , 297.31335,
        298.1449 , 298.34735, 296.99118, 297.50543, 299.01788, 298.82077,
        298.6735 , 296.96567, 298.05334, 298.9269 , 298.4154 , 297.59244,
        299.20117, 298.9539 , 298.05533, 298.11063, 297.6239 , 297.4013 ,
        297.75055, 297.19266, 297.35977, 296.8007 , 298.7328 , 299.46387],
       [298.19818, 299.247  , 298.48346, 298.13217, 298.17703, 298.5722 ,
        298.69025, 298.16904, 297.87875, 298.43475, 299.20438, 298.90155,
        297.9081 , 298.11478, 298.77057, 298.8041 , 297.62128, 297.2444 ,
        298.49738, 298.45856, 298.2292 , 298.1336 , 297.30646, 297.499  ,
        298.34567, 297.73578, 298.17245, 298.16666, 298.6295 , 298.73306],
       [296.37637, 297.3025 , 297.7724 , 297.3715 , 297.36053, 297.26285,
        297.21323, 297.38123, 295.89563, 296.97852, 298.28336, 297.4944 ,
        296.30362, 296.8445 , 298.19333, 298.10297, 296.37622, 296.74197,
        298.38907, 297.98355, 296.82275, 297.14606, 296.04684, 296.17715,
        296.71503, 296.35236, 296.6596 , 296.7834 , 297.7844 , 298.30414],
...
        297.92313, 298.24274, 297.42307, 297.63828, 298.82974, 298.9105 ,
        298.718  , 297.179  , 298.115  , 298.80295, 297.71313, 297.25897,
        298.5432 , 298.52872, 298.2067 , 298.03125, 297.79297, 297.32535,
        297.98065, 297.54904, 297.47748, 297.36523, 298.64145, 299.44977],
       [297.76483, 298.6477 , 298.1786 , 297.91498, 297.98187, 298.17307,
        298.35852, 297.7402 , 297.17886, 297.99335, 298.81607, 298.33807,
        297.3101 , 297.38684, 298.49692, 298.40805, 297.19528, 296.94275,
        298.38297, 298.22678, 297.79565, 297.83344, 296.94675, 297.0327 ,
        297.90106, 297.40228, 297.86707, 298.22717, 298.32672, 298.60068],
       [296.98993, 297.4486 , 297.8563 , 297.33182, 297.4418 , 297.0084 ,
        297.2551 , 297.53455, 296.5322 , 297.2282 , 298.49265, 297.9344 ,
        297.42258, 297.09753, 297.91757, 298.2179 , 297.11145, 296.81232,
        298.53906, 298.24622, 297.43112, 297.4513 , 296.66608, 296.5554 ,
        297.23035, 296.60403, 297.0701 , 296.62405, 298.25665, 298.68585],
       [      nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan]],
      dtype=float32)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 240B 2025-07-01 ... 2025-07-30
  * id          (id) object 104B 'O6uvpzGd5pu' 'fdc6uOvgoji' ... 'at6UHUQatSo'
Attributes: (12/33)
    GRIB_paramId:                             167
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      169
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               K
    long_name:                                mean 2 metre temperature
    units:                                    K
    standard_name:                            unknown
    GRIB_surface:                             0.0
    reduce_dims:                              ['latitude', 'longitude']

The result from `spatial.reduce` is an `xarray` object, which doesn't make much sense for our aggregated data. So instead we convert the results to a Pandas dataframe, which allows us to read the results easier:

In [8]:
agg_temp_df = agg_temp.to_dataframe().reset_index()
agg_temp_df

,id,valid_time,number,t2m
0,O6uvpzGd5pu,2025-07-01,0,297.221893
1,O6uvpzGd5pu,2025-07-02,0,297.939362
2,O6uvpzGd5pu,2025-07-03,0,297.781494
3,O6uvpzGd5pu,2025-07-04,0,297.493011
4,O6uvpzGd5pu,2025-07-05,0,297.603973
...,...,...,...,...
385,at6UHUQatSo,2025-07-26,0,NaN
386,at6UHUQatSo,2025-07-27,0,NaN
387,at6UHUQatSo,2025-07-28,0,NaN
388,at6UHUQatSo,2025-07-29,0,NaN


We see that the aggregated dataframe contains what seems to be kelvin temperature values for each organisation unit and each time period (daily).

### Precipitation

We use the same approach for precipitation by extracting the precipitation or `tp` variable. The main difference here is that we set `how='sum'` since precipitation is typically reported as the total precipitation for an area (not average). 

In [9]:
precip = data_array['tp']
agg_precip = transforms.spatial.reduce(precip, org_units, mask_dim='id', how='sum')
agg_precip_df = agg_precip.to_dataframe().reset_index()
agg_precip_df

,id,valid_time,number,tp
0,O6uvpzGd5pu,2025-07-01,0,0.031982
1,O6uvpzGd5pu,2025-07-02,0,0.045636
2,O6uvpzGd5pu,2025-07-03,0,0.070486
3,O6uvpzGd5pu,2025-07-04,0,0.126077
4,O6uvpzGd5pu,2025-07-05,0,0.126461
...,...,...,...,...
385,at6UHUQatSo,2025-07-26,0,0.000000
386,at6UHUQatSo,2025-07-27,0,0.000000
387,at6UHUQatSo,2025-07-28,0,0.000000
388,at6UHUQatSo,2025-07-29,0,0.000000


We see that the aggregated dataframe contains what seems to be total precipitation values in meters for each organisation unit and each time period (daily). 

## Post-processing

We have now aggregated the temperature and precipitation data to our organisation units. But before we submit the results to DHIS2, we want to make sure they are reported in a format that makes sense to most users. 

For temperature, we convert the data values from kelvin to celcius by subtracting 273.15 from the values:

In [10]:
agg_temp_df['t2m'] -= 273.15
agg_temp_df

,id,valid_time,number,t2m
0,O6uvpzGd5pu,2025-07-01,0,24.071899
1,O6uvpzGd5pu,2025-07-02,0,24.789368
2,O6uvpzGd5pu,2025-07-03,0,24.631500
3,O6uvpzGd5pu,2025-07-04,0,24.343018
4,O6uvpzGd5pu,2025-07-05,0,24.453979
...,...,...,...,...
385,at6UHUQatSo,2025-07-26,0,NaN
386,at6UHUQatSo,2025-07-27,0,NaN
387,at6UHUQatSo,2025-07-28,0,NaN
388,at6UHUQatSo,2025-07-29,0,NaN


For precipitation, to avoid small decimal numbers, we convert the reporting unit from meters to millimeters:

In [11]:
agg_precip_df['tp'] *= 1000
agg_precip_df

,id,valid_time,number,tp
0,O6uvpzGd5pu,2025-07-01,0,31.982395
1,O6uvpzGd5pu,2025-07-02,0,45.636105
2,O6uvpzGd5pu,2025-07-03,0,70.486000
3,O6uvpzGd5pu,2025-07-04,0,126.076889
4,O6uvpzGd5pu,2025-07-05,0,126.460922
...,...,...,...,...
385,at6UHUQatSo,2025-07-26,0,0.000000
386,at6UHUQatSo,2025-07-27,0,0.000000
387,at6UHUQatSo,2025-07-28,0,0.000000
388,at6UHUQatSo,2025-07-29,0,0.000000


## Converting to DHIS2 Format

Before we can send these data to DHIS2, we need to use the `dhsi2eo` utility function `dataframe_to_dhis2_json` to translate each of our aggregated `pandas.DataFrame` into the JSON structure used by the DHIS2 Web API. 

First, for temperature:

In [12]:
agg_temp_json_dict = dataframe_to_dhis2_json(
    df = agg_temp_df,               # aggregated pandas.DataFrame
    org_unit_col = 'id',            # column containing the org unit id
    period_col = 'valid_time',      # column containing the period
    value_col = 't2m',              # column containing the value
    data_element_id = 'VJwwPOOvge6' # id of the DHIS2 data element
)

We can display the first 3 items to see that we have one temperature value for each org unit and period combination.

In [13]:
agg_temp_json_dict['dataValues'][:3]

[{'orgUnit': 'O6uvpzGd5pu',
  'period': '20250701',
  'value': 24.0718994140625,
  'dataElement': 'VJwwPOOvge6'},
 {'orgUnit': 'O6uvpzGd5pu',
  'period': '20250702',
  'value': 24.78936767578125,
  'dataElement': 'VJwwPOOvge6'},
 {'orgUnit': 'O6uvpzGd5pu',
  'period': '20250703',
  'value': 24.631500244140625,
  'dataElement': 'VJwwPOOvge6'}]

And we do the same for precipitation:

In [14]:
agg_precip_json_dict = dataframe_to_dhis2_json(
    df = agg_precip_df,             # aggregated pandas.DataFrame
    org_unit_col = 'id',            # column containing the org unit id
    period_col = 'valid_time',      # column containing the period
    value_col = 'tp',               # column containing the value
    data_element_id = 'eHFmngLqpj4' # id of the DHIS2 data element
)

And inspect the results:

In [15]:
agg_precip_json_dict['dataValues'][:3]

[{'orgUnit': 'O6uvpzGd5pu',
  'period': '20250701',
  'value': 31.98239517211914,
  'dataElement': 'eHFmngLqpj4'},
 {'orgUnit': 'O6uvpzGd5pu',
  'period': '20250702',
  'value': 45.636104583740234,
  'dataElement': 'eHFmngLqpj4'},
 {'orgUnit': 'O6uvpzGd5pu',
  'period': '20250703',
  'value': 70.48600006103516,
  'dataElement': 'eHFmngLqpj4'}]

## Next steps

At this point we have successfully aggregated temperature data in a JSON format that can be used by DHIS2. To learn how to import this JSON data into DHIS2, see [our guide for uploading data values using the Python DHIS2 client](../import-data/using-python-client.ipynb). 